## Imports

In [1]:
# | code-fold: true
# | code-summary: "Load packages"
# | output: false


import os
import numpy as np
import os
import numpy as np
from sympy import Matrix, sqrt
import pytest
from attr import define, field
from sympy import MutableDenseNDimArray as Arr


from zoomy_core.fvm.solver_numpy import Settings
from zoomy_core.model.basemodel import Model
import zoomy_core.model.initial_conditions as IC
import zoomy_core.model.boundary_conditions as BC
from zoomy_core.misc.misc import Zstruct, ZArray
import zoomy_core.misc.misc as misc
import zoomy_firedrake.firedrake_solver as dg


In [2]:
@define(frozen=True, slots=True, kw_only=True)
class SWE(Model):
    dimension: int = 2
    variables: Zstruct = field(init=False)
    aux_variables: Zstruct = field(default=1)
    _default_parameters: dict = field(
        init=False, factory=lambda: {"g": 9.81, "ex": 0.0, "ey": 0.0, "ez": 1.0, "rho": 1000.0, "n": 0.1}
    )
    
    def __attrs_post_init__(self):
        object.__setattr__(self, "variables", self.dimension + 2)
        super().__attrs_post_init__()

    def project_2d_to_3d(self):
        out = ZArray.zeros(6)
        p = self.parameters
        dim = self.dimension
        z = self.position[2]
        b = self.aux_variables[0]
        h = self.variables[1]
        U = [hu / h for hu in self.variables[2 : 2 + dim]]
        out[0] = b
        out[1] = h
        out[2] = U[0]
        out[3] = 0 if dim == 1 else U[1]
        out[4] = 0
        out[5] = p.rho * p.g * h * (1 - z)
        return out

    def flux(self):
        dim = self.dimension
        b = self.variables[0]
        h = self.variables[1]
        U = Matrix([hu / h for hu in self.variables[2 : 2 + dim]])
        g = self.parameters.g
        I = Matrix.eye(dim)
        F = Matrix.zeros(self.variables.length(), dim)
        F[1, :] = h * U.T
        F[2:, :] = h * U * U.T + g / 2 * h**2 * I
        return ZArray(F)
    
    def source(self):
        eps = 1e-16
        dim = self.dimension
        b = self.variables[0]
        h = self.variables[1]
        U = Matrix([hu / h for hu in self.variables[2 : 2 + dim]])
        g = self.parameters.g
        n = self.parameters.n
        abs_u = sqrt(U.dot(U) + eps)
        S = Matrix.zeros(self.n_variables, 1)
        S[2:, 0] = n**2 * g / (h**(1/3) + eps) * U[:, 0] * abs_u
        # S[2:, 0] = n**2 * g  * U[:,0] / h**(1/3)
        return ZArray(S).reshape(self.n_variables,)


        


# Transformation to UFL Code (Medium)

### Map from Sympy to UFL

In [3]:
bcs = BC.BoundaryConditions(
    [
        BC.Extrapolation(tag="wall"),
        BC.Extrapolation(tag="inflow"),
        BC.Extrapolation(tag="outflow"),
    ]
)

 ### Initial condition
def ic_q(x):
    R = 0.15
    r = np.sqrt((x[0] - 0.7)**2 + (x[1] - 0.7)**2)
    b = 0.1*np.sqrt((x[0] - 3.)**2 + (x[1] - 3.)**2)
    return np.array([0.*x[0], np.where(r <= R, 1., 0.9), 0.*x[0], 0.*x[0]])

ic = IC.UserFunction(ic_q)

model = SWE(
    dimension=2,
    boundary_conditions=bcs,
    initial_conditions=ic,
)

settings = Settings(name="Firedrake", output=Zstruct(directory="outputs/firedrake", snapshots=1000, filename='dg', clean_directory=True))


In [8]:
solver = dg.FiredrakeHyperbolicSolver(settings=settings, time_end = 2., CFL=2.0)

In [9]:
main_dir = misc.get_main_directory()
path_to_mesh = os.path.join(main_dir, "meshes", "channel_quad_2d", "mesh_coarse.msh")
solver.solve(path_to_mesh, model)

2025-11-11 12:27:53.500 | INFO     | zoomy_firedrake.firedrake_solver:solve:306 - iteration: 10, time: 0.587626, dt: 0.059405, next write at time: 0.022022
2025-11-11 12:27:53.962 | INFO     | zoomy_firedrake.firedrake_solver:solve:306 - iteration: 20, time: 1.182001, dt: 0.059453, next write at time: 0.042042
2025-11-11 12:27:54.434 | INFO     | zoomy_firedrake.firedrake_solver:solve:306 - iteration: 30, time: 1.776624, dt: 0.059468, next write at time: 0.062062
2025-11-11 12:27:54.621 | INFO     | zoomy_firedrake.firedrake_solver:solve:311 - Finished simulation with in 2.015 seconds
